# 1. 토큰화
## 1-1. casual_tokenize 토큰화, stopwords로 단어 삭제

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import casual_tokenize
import pandas as pd

#nltk.download('stopwords')

df = pd.read_csv('Warframe_reviews.csv')

stop_words = set(stopwords.words('english'))

processed_tokens_list = []

for text in df['review_text']:
    text = str(text)
    
    # 토큰화
    tokens = casual_tokenize(text, reduce_len=True, strip_handles=True)
    
    # nltk.stop_word로 단어 삭제
    filtered_tokens = [t for t in tokens if len(t) > 1 and t.lower() not in stop_words]
    
    processed_tokens_list.append(filtered_tokens)
    
print(processed_tokens_list)

# 각 리뷰의 토큰들을 모두 모으기
all_tokens = [word for tokens in processed_tokens_list for word in tokens]

## 1-2. 품사 태깅
+ game, play 등의 의미없는 단어가 너무 많음. => 일일히 걸러주자
+ 품사 태깅 이유 : 키워드 추출 할때 부사와 같은 꾸며주는 단어가 많이 나와서 추출하는데 의미가 없다고 생각이 들었음
+ 동사, 명사와 같은 의미있는 단어만 뽑아주기 위해 품사 태깅 사용

In [ ]:
# 품사 태깅 테스트 (동사 많이 나온 단어 추출)
# from collections import Counter

# tagged_tokens = nltk.pos_tag(all_tokens)
# nouns = [word for word, pos in tagged_tokens if pos.startswith('VB')]

# counter = Counter(nouns)
# top_30_nn = counter.most_common(30)
# print(top_30_nn)

In [ ]:
# 불용어 품사 정의
stopPos = ['IN', 'CC', 'UH', 'TO', 'MD', 'DT', 'VBZ','VBP']

tagged_tokens = nltk.pos_tag(all_tokens)
print(tagged_tokens)

from collections import Counter

# most_common() : Counter 객체에서 빈도수가 높은 순서대로 데이터를 정렬한 리스트 반환
print(Counter(tagged_tokens).most_common(20))

In [ ]:
# 불용어 처리: 특정 품사 태그에 해당하는 단어만 필터링
stop_words = [word for word, tag in tagged_tokens if tag in stopPos]
filtered_tokens = [token for token in all_tokens if token not in stop_words]

## 1-3. 정규화 (원형 복원)
### 1-3-1. 소문자 정규화

In [ ]:
from matplotlib.pylab import normal

# lower메소드로 정규화 
normalized_tokens = [x.lower() for x in filtered_tokens]
print(normalized_tokens)

### 1-3-2. 어간 추출 (stemming)

In [ ]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
stemmed_tokens = [ps.stem(word) for word in all_tokens]
print(stemmed_tokens)

### 1-3-3. 표제어 추출

In [ ]:
# nltk.download('wordnet')
# nltk.download('omw-1.4')  # 선택: 시소러스(동의어) 관련

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]
print(lemmatized_tokens)

## 1-4. 불용어 처리

In [ ]:
stop_text = ['game', 'warframe', 'destiny', 'destiny2', 'thefirstdescendant', 'gameplay', 'descendant', 'tfd', 'first', 'play']

filtered_tokens = [tokens for tokens in lemmatized_tokens if tokens not in stop_text]

# 2. 임베딩

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Jina-Embeddings-v3 모델 로드
model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)

# 사용할 태스크 설정 (예: 문장 유사도 계산을 위한 임베딩)
task = "text-matching"  

df = pd.read_csv('The_first_descendant_positive_reviews.csv')
df['review_text'] = df["review_text"].fillna("").astype(str)


# review_text 열을 리스트로 변환
sentences = filtered_tokens

# 문장 임베딩 생성
embeddings = model.encode(
    sentences,
    task=task,         # 태스크 지정 (검색용 쿼리 임베딩)
    prompt_name=task,   # 프롬프트 지정
    device="cpu"
)

# 결과 출력
print(embeddings.shape)  # 예: (3, 768) -> 3개의 문장이 768차원 벡터로 변환됨